In [ ]:
import numpy as np
from nm_lib import nm_lib as nm
import threading

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from matplotlib import animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

from scipy.optimize import curve_fit

import time

# Space-time partial differential equation: Study of the diffusive equation (implicit methods)

Let's consider now the viscous term in Burger's equation: 

$$\frac{\partial u}{\partial t} = \nu \frac{\partial^2 u}{\partial x^2}, \tag{1}$$

## 1- Apply an explicit method. 

What would be the CFL condition for a viscous term where $\nu$ is either a constant or an array that depends on $x$. We would like to solve equation (1) numerically for $x  [x_0, x_f]$ with $x_0 = −2.6$, $x_f = 2.6$, periodic boundary conditions and with the initial condition:

$$u(x,t=t_0) = A\exp(-(x-x_0)^2/W^2)   \tag{2}$$

with $A=0.3$, $W=0.1$, and $x_0=0$. __Suggestion__: Apply the first derivative upwind and the second downwind. Apply Von Newman analysis. Is it stable? What is the time-step dependence with $\Delta x$? 

How many steps are needed to reach a $t=1.8$ for $nump=128$? And $256$? 

In [ ]:
def u(x, A, W):
    x0 = 0
    return A * np.exp(-(-x-x0)**2/W**2)

x0 = -2.6
xf = 2.6

def get_xx(x0, xf, nump):
    return np.arange(nump)/(nump-1.0) * (xf-x0) + x0

A = 0.3
W = 0.1

The viscous term using Forward-Euler in time and First upwind then downwind in space:

\begin{align}
\frac{u_j^{n+1}-u_j^n}{\Delta t} &= \nu \frac{(u_{j+1}^n - 2u_j^n + u_{j-1}^n)}{\Delta x^2}.\\
\end{align}

Now Von Neumann analysis of the viscous term, first assume peculiar solution on the form

$$
u_j^n = \xi^n e^{ikj\Delta x}.
$$

Inserting this in the equation above gives
\begin{align}
e^{ikj\Delta x}\frac{\xi^{n+1}-\xi^{n}}{\Delta t} &= \nu \xi^n e^{ikj\Delta x} \frac{e^{ik\Delta x}-2+e^{-ik\Delta x}}{\Delta x^2}\\
\frac{\xi^{n+1}-\xi^{n}}{\xi^n} &=   \frac{\nu\Delta t}{\Delta x^2}\left(e^{ik\Delta x}-2+e^{-ik\Delta x}\right)\\
\xi &=\frac{\nu\Delta t}{\Delta x^2}\left(\cos{(k\Delta x)}+i\sin{(k\Delta x)} -2+\cos{(k\Delta x)-i\sin{(k\Delta x)}} \right) +1\\
\xi &=\frac{2\nu\Delta t}{\Delta x^2}\left(\cos{(k\Delta x)} -1 \right) +1\\
\xi &=-\frac{4\nu\Delta t}{\Delta x^2}\sin^2{\left(\frac{k\Delta x}{2}\right)} + 1
\end{align}

We require that $|\xi^n|^2\leq 1$ for all $k$. This gives

\begin{align}
\left|-\frac{4\nu\Delta t}{\Delta x^2}\sin^2{\left(\frac{k\Delta x}{2}\right)} + 1 \right|^2 &\leq 1\\
\frac{16\nu^2\Delta t^2}{\Delta x^4}\sin^4{\left(\frac{k\Delta x}{2}\right)} - \frac{4\nu\Delta t}{\Delta x^2}\sin^2{\left(\frac{k\Delta x}{2}\right)} + 1 &\leq 1
\end{align}

This needs to hold for all $k$, so we look at the max of the sine function

$$
\frac{16\nu^2\Delta t^2}{\Delta x^4} - \frac{4\nu\Delta t}{\Delta x^2} + 1 \leq 1
$$

\begin{align}
\frac{16\nu^2\Delta t^2}{\Delta x^4} &\leq \frac{4\nu\Delta t}{\Delta x^2}\\
\frac{4\nu \Delta t}{\Delta x^2} &\leq 1
\end{align}

$$
\Delta t \leq \frac{\Delta x^2}{4\nu}
$$

In [ ]:
xx = get_xx(x0, xf, nump=256)
hh = u(xx, A, W)
nt = 30
dt = 1.1
a = 1

tt_diff, unnt_diff = nm.evolv_diff_burgers(xx, hh,nt, a, cfl_cut = 0.98)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
def init(): 
    axes.plot(xx,unnt_diff[0,:])
    axes.set_ylim(-0.05, 0.32)
    axes.grid(True)

def animate(i):
    axes.clear()
    axes.plot(xx,unnt_diff[i,:])
    axes.set_ylim(-0.05, 0.32)
    axes.set_title('t={:.2f}, i={:g}'.format(tt_diff[i],i))
    axes.grid(True)

anim = FuncAnimation(fig, animate, interval=50, frames=nt, init_func=init)
HTML(anim.to_jshtml())

Looks good now. A little diffusive.

<span style="color:green">JMS</span>.

<span style="color:blue">fixed! </span>.


In [ ]:
dt = 1e-2
t, unnt, errt, countt = nm.Newton_Raphson(xx, hh, a, dt, nt, toll= 1e-5, ncount=30, bnd_type='wrap', bnd_limits=[1,1])

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
def init(): 
    axes.plot(xx,unnt[:,0])
    axes.set_ylim(-0.05, 0.32)
    axes.grid(True)

def animate(i):
    axes.clear()
    axes.plot(xx,unnt[:,i])
    axes.set_ylim(-0.05, 0.32)
    axes.set_title('t={:.2f}, i={:g}'.format(t[i],i))
    axes.grid(True)

anim = FuncAnimation(fig, animate, interval=50, frames=nt, init_func=init)
HTML(anim.to_jshtml())

Now we see the Newton Rapson method. This looks much more diffusive but when we look at the time compared to diff-burgers we can see that for the diff-burgers the timesteps are so small that we are at 0.00s still with 2 significant digits because the dt become so small. With the newton rapson we can increment with much larger timesteps.

<span style="color:green">JMS</span>.

<span style="color:blue"> Ok </span>.

## Choose one of the following options: 

## 2- Implicit methods.

In the [wiki](https://github.com/AST-Course/AST5110/wiki/Implicit-methods), we describe some implicit or semi-explicit methods that allow relaxing the CFL constraint on diffusive terms. Consider Newton-Rapson method and repeat the previous numerical experiment. For this, you will need to implement the following   


$F_j = u^{n+1}_j - u^n_j - \nu \, (u^{n+1}_{j+1} - 2u^{n+1}_{j}+u^{n+1}_{j-1})\frac{\Delta t}{\Delta x^2}$

in `NR_f` and `step_diff_burgers` functions in `nm_lib`. 

And the Jacobian can be easily built. 

$J(j,k) = F_j'(u^{n+1}_k)$

fill in the `jacobian` function in `nm_lib`. Note that this matrix is linear with $u$. 

Test the model with [wiki](https://github.com/AST-Course/AST5110/wiki/Self-similar-solution-for-parabolic-eq) self-similar solutions. How long it takes each time step compared to the Lax-method? Use `time.time` library. Do it for nump=256, nt=30 and dt = 0.1.

In [ ]:
xx = get_xx(x0, xf, nump=256)
hh = u(xx, A, W)
nt = 30
dt = 0.01
a = 1

n_times = 10

diff_times = np.zeros(n_times)
for i in range(n_times):
    start = time.time()
    nm.evolv_diff_burgers(xx, hh,nt, a, cfl_cut = 0.98)
    end = time.time()
    diff_times[i] = end-start

nr_times = np.zeros(n_times)
for i in range(n_times):
    start = time.time()
    nm.Newton_Raphson(xx, hh, a, dt, nt, toll= 1e-5, ncount=2, bnd_type='wrap', bnd_limits=[1,1])
    end = time.time()
    nr_times[i] = end - start
    
mean_diff = np.mean(diff_times)
std_diff = np.std(diff_times)

mean_nr = np.mean(nr_times)
std_nr = np.std(nr_times)

In [ ]:
print("Time diff: {:.1e}+-{:.1e}".format(mean_diff, std_diff))
print("Time NR: {:.1e}+-{:.1e}".format(mean_nr, std_nr))

Evolv_diff_burgers is two orders of magnitude slower than NR.

<span style="color:green">JMS</span>.

<span style="color:blue"> fixed.  </span>.

In order to test the simulation, use `curve_fit` from `scipy.optimize`. 

__hint__ consider to use a good initial guess (`p0`) in and `bnd_limits` to facilitate the fitting wiht `curve_fit`. What happens to the solution when increasing dt? How much can be improved in limiting the tolerance?

The self similar solution says that the height of the gaussian should go like $t^{-1/2}$ and the width should go like $t^{1/2}$.

In [ ]:
def gaussian(x, mu, sigma, norm):
    p = np.exp(-np.power(x - mu, 2.) / (2 * np.power(sigma, 2.)))
    p = p*norm
    return p

In [ ]:
t, unnt, errt, countt = nm.Newton_Raphson(xx, hh, a, dt, nt, toll= 1e-5, ncount=15, bnd_type='wrap', bnd_limits=[1,1])

In [ ]:
import warnings
from scipy.optimize import OptimizeWarning

In [ ]:
peaks = np.zeros(len(t))
fwhms = np.zeros(len(t))

for i in range(len(t)):
    popt, pcov = curve_fit(gaussian, xx, unnt[:,i], p0=[0, 0.3, 0.3])
    g = gaussian(xx, *popt)
    fwhms[i] = popt[1]
    peaks[i] = np.max(g)

In [ ]:
def log_slope_and_fit(x_array, y_array):
    # Exclude the first value in both x_array and y_array
    # because x[0]=0 => logspace = -inf
    x_array_excluded = x_array[1:]
    y_array_excluded = y_array[1:]

    # Convert the x and y arrays into log space
    log_x = np.log10(x_array_excluded)
    log_y = np.log10(y_array_excluded)

    # Fit a polynomial of degree 1 (linear fit) to the log-log data
    coeffs = np.polyfit(log_x, log_y, 1)

    # The first coefficient in the output is the slope of the linear fit
    slope = coeffs[0]

    # Create the fitted y values in log space using the coefficients
    log_y_fit = np.polyval(coeffs, log_x)

    # Convert the x_fit and y_fit arrays back to the original space
    x_fit = 10 ** log_x
    y_fit = 10 ** log_y_fit

    return slope, x_fit, y_fit

In [ ]:
slope_fwhm, x_fit_fwhm, y_fit_fwhm = log_slope_and_fit(t, fwhms)
slope_peaks, x_fit_peaks, y_fit_peaks = log_slope_and_fit(t, peaks)

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].loglog(t, fwhms, label="FWHMs")
ax[0].loglog(x_fit_fwhm, y_fit_fwhm, linestyle="--", label="Polynomial fit")
ax[1].loglog(t, peaks, label="Peaks")
ax[1].loglog(x_fit_peaks, y_fit_peaks, linestyle="--", label="Polynomial fit")
ax[0].legend()
ax[1].legend()

In [ ]:
print(f"Slope FWHMs:{slope_fwhm}")
print(f"Slope Peaks:{slope_peaks}")

We can see that the FWHM goes as $t^{1/2}$ and the peaks as $t^{-1/2}$ when doing a Gaussian fit on Newton-Rapson.

<span style="color:green">JMS</span>.

<span style="color:blue"> Ok  </span>.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
def init(): 
    axes.set_ylim(-0.05, 0.32)
    axes.plot(xx,unnt[:,0], label="NR")
    popt, pcov = curve_fit(gaussian, xx, unnt[:,0], p0=[0, 0.3, 0.3])
    axes.plot(xx, gaussian(xx, *popt), label="Gaussian fit", linestyle="--")
    axes.grid(True)
    axes.legend()

def animate(i):
    axes.clear()
    axes.plot(xx,unnt[:,i], label="NR")
    axes.set_ylim(-0.05, 0.32)
    popt, pcov = curve_fit(gaussian, xx, unnt[:,i], p0=[0, 0.3, 0.3])
    axes.plot(xx, gaussian(xx, *popt), label="Gaussian fit", linestyle="--")
    axes.grid(True)
    axes.legend()
    
anim = FuncAnimation(fig, animate, interval=50, frames=nt, init_func=init)
HTML(anim.to_jshtml())

We can see that the Gaussian follows the Newton-Rapson solution as the self-similar solution says.

<span style="color:green">JMS</span>.

<span style="color:blue"> Good  </span>.


----------------------------------------------

Let's consider a non-linear function where $\nu$ depends on $u$. To keep it simple, solve the following: 

$\frac{\partial u}{\partial t} = u \frac{\partial^2 u}{\partial x^2}$

where $\nu_0$ is a constant and the same initial conditions as the previous exercise (fill in `Newton_Raphson_u`, `jacobian_u` and `NR_f_u`. Consider an error limit of $10^{-4}$ and compare the previous exercise (with the same error limit). How many iterations needs now the method to converge to the right solution? Why? Increase `ncount` to 1000. 

In [ ]:
dt = 0.1
t, unnt, errt, countt_NR_u = nm.Newton_Raphson_u(xx, hh, dt, nt, toll= 1e-4\
                                                 , ncount=1000, bnd_type='wrap', bnd_limits=[1,1])
t, unnt, errt, countt_NR = nm.Newton_Raphson(xx, hh, a, dt, nt, toll= 1e-4\
                                               , ncount=1000, bnd_type='wrap', bnd_limits=[1,1])

In [ ]:
print(countt_NR_u)
print(countt_NR)

In [ ]:
t, unnt, errt, countt_NR_u = nm.Newton_Raphson_u(xx, hh, dt, nt, toll= 1e-6\
                                                 , ncount=1000, bnd_type='wrap', bnd_limits=[1,1])
t, unnt, errt, countt_NR = nm.Newton_Raphson(xx, hh, a, dt, nt, toll= 1e-6\
                                               , ncount=1000, bnd_type='wrap', bnd_limits=[1,1])

In [ ]:
print(countt_NR_u)
print(countt_NR)

We can see that the NR_u find a solution in much fewer iterations. And that the NR has much more iterations, sometimes even maxing out. Lowering the tolerance does not seem to affect NR_u, but makes NR max out the iterations every time meaning that it does not converge to a solution before it stops.

<span style="color:green">JMS</span>.

<span style="color:blue"> Good job!  </span>.

## 3- Semi-explicit methods. 

__a)__ Super-time-stepping (STS) schemes work for parabolic terms. STS is an API method that performs a subset of "unstable" intermediate steps, but the sum of all the steps is stable. Visualize how `taui_sts` varies with $nu$ and $niter$. Compare the solution with the analytical one for the final and intermediate STS steps. For the full STS steps, how improves the solution with $nu$? and $niter$? Is there a relation between the error and these two parameters, $nu$, and $niter$? For which $niter$ and $nu$ the method provides larger steps than an ordinary explicit. For this exercise, fill in `evol_sts`, and `taui_sts`. 